In [1]:
# Basic python packages
import os
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict
import glob

# General machine learning packages
from sklearn.model_selection import train_test_split

# Packages related to images
from PIL import Image
import PIL


In [8]:
#Paths to different folders/files
image_dir = "../Data/Rijksmuseum/jpg2"
test_image_dir = "../Data/Rijksmuseum/test_set"
labels_file = "../Data/Rijksmuseum/xml_files_no_anons.csv"
training_path = "../Data/Rijksmuseum/training_data/"
validation_path = "../Data/Rijksmuseum/validation_data/"

img_size = (200, 200) #Size of the input of the neural networks
IMG_SHAPE = img_size + (3,)
batch_size = 32
n_labels = 6622

In [9]:
# #Paths to different folders/files
# image_dir = "../Data/Rijksmuseum/jpg2"
# resized_img_dir = "..Data/Rijksmuseum/resized_jpg2"

# train_dir = "../Data/Rijksmuseum/train_set"
# test_image_dir = "../Data/Rijksmuseum/test_set"

# training_path = "../Data/Rijksmuseum/training_data/"
# validation_path = "../Data/Rijksmuseum/validation_data/"

# labels_file = "../Data/Rijksmuseum/xml_files.csv"

# img_size = (200, 200) #Size of the input of the neural networks
# IMG_SHAPE = img_size + (3,)
# batch_size = 32
# n_labels = 6620

In [10]:
labels = pd.read_csv(labels_file)
# labels[['Identifier', 'Creator']]

labels = labels[['Identifier', 'Creator']]
# subs = subs.set_index('Identifier')
labels

,Identifier,Creator
0,SK-A-4878,"Everdingen, Caesar Boëtius van"
1,SK-A-4877,"Maris, Matthijs"
2,SK-A-4881,"Maes, Nicolaes"
3,RP-P-1992-35,"Coornhert, Dirck Volckertsz"
4,RP-P-1992-36,"Coornhert, Dirck Volckertsz"
...,...,...
94663,RP-P-2011-173-1,"Fellmann, M."
94664,RP-T-1886-A-704-26,"Bramer, Leonaert"
94665,RP-P-OB-86.512,"Bos, Maarten"
94666,NG-NM-8358,"Coenraads, Jacobus (Senior)"


In [12]:
len(labels.Creator.unique())

6620

# Image size
Most machine learning requires that the input is always of the same size. Because our images are not always of the same size. We have to resize them

In [17]:
def resize_images(img_names, img_dir, new_img_dir):
    #Deprecated, replaced by the flow from directory
    for img in img_names:
        Image.open(img_dir + "/" + img + ".jpg").resize(img_size).save(new_img_dir + "/" + img + ".jpg")

resize_images(labels["Identifier"], image_dir, resized_train_dir)

# Split

In [21]:
X_train, X_test, y_train, y_test = train_test_split(labels["Identifier"].to_numpy(), labels["Creator"].to_numpy(), test_size=0.2, random_state=42)
X_train

array(['RP-P-BI-250', 'RP-P-BI-6678', 'RP-P-1880-A-4126', ...,
       'RP-P-1944-663', 'SK-C-221', 'RP-P-1918-2049'], dtype=object)

# Reformat data for learning
To be able to load the data with a generator. We must split the training and validation data and place them into a folder based on their label.

In [ ]:
def generate_label_folders(image_path, image_names, image_labels, destination_path):
    """
    Splits a single folder with images into multiple folders where images are placed based on their labels.

    :image_path: path to the folder with the images
    :image_names: A numpy array with the names of all images
    :image_labels: A numpy array with the labels of all images
    :destination_path: Path of the folder where the images are placed into
    :return: Nothing
    """ 
    for i in range(len(image_names)):
        # Check if the directory exists. Else, make one
        isExist = os.path.exists(destination_path + str(image_labels[i]))
        if not isExist:
            os.makedirs(destination_path + str(image_labels[i]))
            
        # Copy the image
        img = Image.open(image_path + "/" + image_names[i] + ".jpg")
        img.save(destination_path + "/" + str(image_labels[i]) + "/" + image_names[i] + ".jpg")
        

# generate_label_folders(image_dir, X_train, y_train, training_path)
# generate_label_folders(image_dir, X_test, y_test, test_image_dir)